![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSWjxMNIJ4R9GKeADLyBVJYy4uq-_QjN_eF8A&usqp=CAU)mysnakepet.com

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import cv2
import random
from random import randint
import time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Codes from Kamal Das https://www.kaggle.com/kmldas/beginner-s-guide-image-augmentation-transforms

In [ ]:
import torch
from torch.utils.data import Dataset, random_split, DataLoader
import torch.nn.functional as F
import torch.nn as nn

from PIL import Image
from scipy import ndimage

import torchvision
import torchvision.models as models
import torchvision.transforms as T
from torchvision.utils import make_grid
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder

from tqdm.notebook import tqdm

from sklearn.metrics import f1_score

#We don't have submission/labels/path images 

In [ ]:
DATA_DIR = '../input/identify-snake-breed-hackerearth/dataset'


TRAIN_DIR = DATA_DIR + '/train'                           
#TEST_DIR = DATA_DIR + '/test'                             

#TRAIN_CSV = DATA_DIR + '/labels.csv'                     
#TEST_CSV = DATA_DIR + '/submission.csv'
#TRAIN_DIR = DATA_DIR + '/images'

In [ ]:
data_df = pd.read_csv('../input/identify-snake-breed-hackerearth/dataset/train.csv', encoding='utf8')
data_df.head()

In [ ]:
labels_names=data_df["breed"].unique()
labels_sorted=labels_names.sort()

labels = dict(zip(range(len(labels_names)),labels_names))
labels 

#To use numbers instead of names for labels. Lets add the numbers as labels to the dataframe

In [ ]:
lbl=[]
for i in range(len(data_df["breed"])):
    temp=list(labels.values()).index(data_df.breed[i])
    lbl.append(temp)

    
data_df['image_id'] = lbl
#data_df['lbl'] = data_df['lbl'].astype(str)
data_df.head()

#EDA

In [ ]:
num_images = len(data_df["image_id"])
print('Number of images in Training file:', num_images)
no_labels=len(labels_names)
print('Number of snakes breeds in Training file:', no_labels)

In [ ]:
bar = data_df["breed"].value_counts(ascending=True).plot.barh(figsize = (30,120))
plt.title("Distribution of the Snake Breeds", fontsize = 20)
bar.tick_params(labelsize=16)
plt.show()

In [ ]:
data_df["breed"].value_counts(ascending=False)

#Add the path of each image to the file. We don't have path to add.

In [ ]:
#path_img=[]
#for i in range(len(data_df["breed"])):
 #   temp=TRAIN_DIR + "/" + str(data_df.breed[i]) + ".jpg"
  #  path_img.append(temp)

#data_df['path_img'] =path_img
#data_df.head()

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision.all import *
from fastai.imports import *
from fastai.vision.data import *
from fastai import *
import numpy as np
import fastai
import matplotlib.pyplot as plt

In [ ]:
path = Path("/kaggle/input/identify-snake-breed-hackerearth/dataset/train")
path.ls()

In [ ]:
np.random.seed(42)
data = ImageDataLoaders.from_folder(path, train=".", valid_pct=0.2, item_tfms=RandomResizedCrop(512, min_scale=0.75),
                                    bs=32,batch_tfms=[*aug_transforms(size=256, max_warp=0), Normalize.from_stats(*imagenet_stats)],num_workers=0)

In [ ]:
data.show_batch(nrows=4, figsize=(7,8))

In [ ]:
data.show_batch(nrows=3, figsize=(7,8))

In [ ]:
data.show_batch(nrows=2, figsize=(7,8))

In [ ]:
data.show_batch(nrows=1, figsize=(7,8))

In [ ]:
#path = Path('/kaggle/input//')
dls = ImageDataLoaders.from_folder(path, train='train',
                                   item_tfms=Resize(224),valid_pct=0.2,
                                   bs=64,seed=0)
dls.show_batch()

In [ ]:
def _add1(x): return x+1
dumb_tfm = RandTransform(enc=_add1, p=0.5)
start,d1,d2 = 2,False,False
for _ in range(40):
    t = dumb_tfm(start, split_idx=0)
    if dumb_tfm.do: test_eq(t, start+1); d1=True
    else:           test_eq(t, start)  ; d2=True
assert d1 and d2
dumb_tfm

In [ ]:
from PIL import Image

img = Image.open("../input/identify-snake-breed-hackerearth/dataset/train/0002023e6f.jpg")
img

In [ ]:
_,axs = subplots(1,2)
show_image(img, ctx=axs[0], title='original')
show_image(img.flip_lr(), ctx=axs[1], title='flipped');

In [ ]:
_,axs = plt.subplots(1,3,figsize=(12,4))
for ax,sz in zip(axs.flatten(), [300, 500, 700]):
    show_image(img.crop_pad(sz), ctx=ax, title=f'Size {sz}');

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Marília Prata, @mpwolke Was here' )